# Simulação e Agrupamento de Corridas da F1 com o Modelo de Mallows

Neste notebook, simulamos resultados de 30 corridas geradas aleatoriamente e aplicamos
um algoritmo baseado no Modelo de Mallows para agrupar rankings parciais.

O objetivo é:
- Gerar corridas artificiais que sigam diferentes padrões de desempenho.
- Aplicar o algoritmo MCMC (baseado no Algorithm 4 do artigo `Probabilistic preference learning with the Mallows rank model`)
  para estimar quais corridas pertencem a quais grupos (clusters).

In [20]:
#ALGORITHM 4 — MCMC for Clustering Partial Rankings (from the article)
#
#Input:
#    - Observed rankings π₁, ..., π_N (possibly partial)
#    - Number of clusters C
#    - Initial parameters {ρ_c, α_c, z_j}
#
#Repeat for each MCMC iteration:
#    1. (Gibbs step) Update cluster assignments z_j:
#        For each ranking π_j:
#            Compute P(z_j = c | π_j, ρ_c, α_c) ∝ exp(-α_c * d(π_j, ρ_c))
#            Sample z_j from this distribution.
#
#    2. (Metropolis–Hastings) Update consensus rankings ρ_c:
#        For each cluster c:
#            Propose new ρ_c' using "leap-and-shift" move.
#            Accept or reject based on posterior probability ratio.
#
#    3. (Optional) Update dispersion parameters α_c:
#        Sample α_c from posterior given cluster assignments.
#
#Until convergence.
#
#Output:
#    - Estimated consensus rankings ρ_c
#    - Cluster assignments z_j
#    - Dispersion parameters α_c

In [21]:
import numpy as np
import random
from collections import Counter

A distância de Kendall Tau mede o número de pares de itens que estão em ordem diferente entre dois rankings.

No contexto do modelo de Mallows, ela representa o quanto um ranking observado π_j difere do ranking de consenso ρ_c.

In [22]:
def kendall_distance(r1, r2):
    common_items = [x for x in r1 if x in r2]
    pairs = 0
    n = len(common_items)
    for i in range(n):
        for j in range(i + 1, n):
            a1, a2 = common_items[i], common_items[j]
            if a1 in r2 and a2 in r2:
                if (r1.index(a1) - r1.index(a2)) * (r2.index(a1) - r2.index(a2)) < 0:
                    pairs += 1
    return pairs

Aqui simulamos as corridas da F1.

Usamos dois clusters diferentes:
- Cluster 1 (pistas rápidas): pilotos como VER e NOR no topo.
- Cluster 2 (pistas técnicas): pilotos como ALO e STR no topo.

Cada corrida é gerada aplicando pequenas permutações (ruído) ao ranking de consenso ρ_c.

In [23]:
def generate_mallows_centered(center, alpha, n_samples, top_k=None):
    n = len(center)
    rankings = []
    for _ in range(n_samples):
        perm = center.copy()
        n_swaps = np.random.poisson(alpha)
        for _ in range(n_swaps):
            i, j = np.random.choice(range(n), 2, replace=False)
            perm[i], perm[j] = perm[j], perm[i]
        if top_k:
            perm = perm[:top_k]
        rankings.append(perm)
    return rankings

# Parâmetros
pilotos = ["VER", "NOR", "LEC", "HAM", "RUS", "PIA", "SAI", "PER", "ALO", "STR"]
n_corridas = 30
top_k = 8

# Dois padrões de consenso
consenso_cluster1 = pilotos.copy()     # pilotos rápidos
consenso_cluster2 = pilotos[::-1]      # pilotos técnicos

# Gerar corridas
rankings_cluster1 = generate_mallows_centered(consenso_cluster1, alpha=2, n_samples=15, top_k=top_k)
rankings_cluster2 = generate_mallows_centered(consenso_cluster2, alpha=2, n_samples=15, top_k=top_k)
rankings = rankings_cluster1 + rankings_cluster2
true_labels = [0] * 15 + [1] * 15  # rótulos verdadeiros

In [24]:
print("Corridas simuladas (Rankings π_j)")
for i, r in enumerate(rankings):
    cluster_real = "Cluster 1 (pistas rápidas)" if true_labels[i] == 0 else "Cluster 2 (pistas técnicas)"
    print(f"Corrida {i+1:02d} | {cluster_real}: {' > '.join(r)}")

Corridas simuladas (Rankings π_j)
Corrida 01 | Cluster 1 (pistas rápidas): VER > NOR > LEC > HAM > RUS > PIA > SAI > PER
Corrida 02 | Cluster 1 (pistas rápidas): HAM > NOR > VER > SAI > RUS > ALO > LEC > PER
Corrida 03 | Cluster 1 (pistas rápidas): VER > NOR > LEC > HAM > RUS > PIA > SAI > PER
Corrida 04 | Cluster 1 (pistas rápidas): NOR > STR > HAM > LEC > SAI > PIA > RUS > PER
Corrida 05 | Cluster 1 (pistas rápidas): RUS > NOR > LEC > HAM > VER > PIA > SAI > PER
Corrida 06 | Cluster 1 (pistas rápidas): SAI > RUS > LEC > HAM > NOR > PIA > VER > PER
Corrida 07 | Cluster 1 (pistas rápidas): PER > NOR > LEC > HAM > RUS > PIA > SAI > VER
Corrida 08 | Cluster 1 (pistas rápidas): VER > NOR > LEC > STR > RUS > PIA > ALO > SAI
Corrida 09 | Cluster 1 (pistas rápidas): PIA > NOR > PER > SAI > RUS > VER > HAM > ALO
Corrida 10 | Cluster 1 (pistas rápidas): HAM > NOR > STR > VER > ALO > PIA > SAI > PER
Corrida 11 | Cluster 1 (pistas rápidas): VER > NOR > LEC > HAM > RUS > STR > SAI > PER
Corrida 1

Dado um conjunto de rankings em um cluster, obtemos o ranking de consenso ρ_c
contando a frequência com que cada piloto aparece nas primeiras posições.

In [25]:
def consensus_ranking(rankings, all_items):
    scores = Counter()
    for r in rankings:
        for pos, item in enumerate(r):
            scores[item] += len(all_items) - pos
    ordered = [x for x, _ in scores.most_common()]
    return ordered

Agora aplicamos as etapas principais do Algorithm 4 (MCMC Clustering Mallows).

1. Atualiza as atribuições z_j (qual corrida pertence a qual cluster).
2. Atualiza os rankings de consenso ρ_c com base nas corridas atribuídas.

In [26]:
C = 2  # número de clusters
assignments = np.random.choice(C, n_corridas)
clusters = {c: [] for c in range(C)}

for iteration in range(50):  # iterações MCMC
    clusters = {c: [] for c in range(C)}

    # Atualiza atribuições z_j
    for i, r in enumerate(rankings):
        distances = []
        for c in range(C):
            if clusters[c]:
                rho = consensus_ranking(clusters[c], pilotos)
                d = kendall_distance(r, rho)
            else:
                d = np.random.rand()
            distances.append(d)
        new_cluster = np.argmin(distances)
        assignments[i] = new_cluster
        clusters[new_cluster].append(r)

    # Atualiza rankings de consenso ρ_c
    consensos = []
    for c in range(C):
        if clusters[c]:
            rho = consensus_ranking(clusters[c], pilotos)
            consensos.append(rho)
        else:
            consensos.append(random.sample(pilotos, len(pilotos)))

In [27]:
print("\nResultados do MCMC")
for c, rho in enumerate(consensos):
    print(f"\nCluster {c + 1} - Ranking de consenso estimado:")
    print(" → ", " > ".join(rho[:top_k]))


Resultados do MCMC

Cluster 1 - Ranking de consenso estimado:
 →  VER > NOR > LEC > HAM > RUS > PER > PIA > SAI

Cluster 2 - Ranking de consenso estimado:
 →  SAI > STR > ALO > HAM > PER > PIA > RUS > LEC


In [29]:
print(assignments)

[0 1 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1]


# Conclusões

- O vetor `assignments` contém as atribuições finais de cada corrida:
  indica a qual cluster (tipo de corrida) o modelo acredita que ela pertence.

- Cada cluster tem um ranking de consenso `ρ_c` estimado,
  que representa o padrão de desempenho médio dos pilotos naquele grupo.

- Mesmo sem conhecer os rótulos verdadeiros, o algoritmo
  consegue agrupar corridas com padrões de resultados semelhantes,
  reproduzindo a ideia do Modelo de Mallows para ranking clustering.